In [1]:
# Connecting with Spotify API
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Initializing spotipy with Spotify API credentials
client_id = '5093ad0ad45f435ca7ce43b2afd406b9'  # Replace with your client ID
client_secret = '75187bd4618948b49090ebc2c9b57eca'  # Replace with your client secret
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
# Fetching data from Spotify Million Playlist dataset
import json

# Load JSON data from file
with open('data/challenge_set.json', 'r') as file: # Replace with local dataset path
    data = json.load(file)

# Initialize an empty list to collect all track data
all_tracks = []

# Loop through each playlist in the dataset
for playlist in data['playlists']:
    for track in playlist['tracks']:
        # Add playlist-level information to each track record
        track_info = {
            'playlist_name': playlist.get('name', 'Unknown'),
            'playlist_pid': playlist['pid'],
            'playlist_num_tracks': playlist['num_tracks'],
            'track_pos': track['pos'],
            'artist_name': track['artist_name'],
            'track_uri': track['track_uri'],
            'artist_uri': track['artist_uri'],
            'track_name': track['track_name'],
            'album_uri': track['album_uri'],
            'duration_ms': track['duration_ms'],
            'album_name': track['album_name']
        }
        all_tracks.append(track_info)

# Convert the list of track dictionaries to a DataFrame
df_spotify = pd.DataFrame(all_tracks)

# Display the first few rows of the DataFrame to verify
print(df_spotify.head())

  playlist_name  playlist_pid  playlist_num_tracks  track_pos artist_name   
0         Party       1000000                   75          0   AronChupa  \
1         Party       1000000                   75          1   AronChupa   
2         Party       1000000                   75          2       Lorde   
3         Party       1000000                   75          3       Lorde   
4         Party       1000000                   75          4       Lorde   

                              track_uri   
0  spotify:track:66U0ASk1VHZsqIkpMjKX3B  \
1  spotify:track:5MhsZlmKJG6X5kTHkdwC4B   
2  spotify:track:0GZoB8h0kqXn7XFm4Sj06k   
3  spotify:track:35kahykNu00FPysz3C2euR   
4  spotify:track:3G6hD9B2ZHOsgf4WfNu7X1   

                              artist_uri   
0  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh  \
1  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh   
2  spotify:artist:163tK9Wjr9P9DmM0AVK7lm   
3  spotify:artist:163tK9Wjr9P9DmM0AVK7lm   
4  spotify:artist:163tK9Wjr9P9DmM0AVK7lm   

             

In [3]:
# Creating new dataset for getting lyrics of unique songs
df_lyrics = df_spotify.drop_duplicates(subset=['track_uri']).drop(['playlist_name', 'playlist_pid', 'playlist_num_tracks', 'track_pos', 'artist_name',
       'artist_uri', 'track_name', 'album_uri', 'duration_ms', 'album_name'], axis=1).reset_index(drop=True)

In [ ]:
df_lyrics.head()

In [ ]:
sp.artist('spotify:artist:5vCOdeiQt9LyzdI87kt5Sh')

In [ ]:
def get_artist_and_track_name(track_uri):
    try:
        track_data = sp.track(track_uri)
        artist_name = track_data['artists'][0]['name'] if track_data['artists'] else 'Unknown Artist'
        #track_name = track_data['name'] if track_data else 'Unknown Track'
        return artist_name#, track_name
    except Exception as e:
        print(f"Error retrieving track details: {str(e)}")
        return 'Error', 'Error'

In [ ]:
track_uri = 'spotify:track:66U0ASk1VHZsqIkpMjKX3B' 
artist_name, track_name = get_artist_and_track_name(track_uri)
print(f"Artist: {artist_name}, Track: {track_name}")

In [ ]:
# Function to fetch artist name and track name
def get_track_details(uri):
    try:
        track = sp.track(uri)
        artist_name = track['artists'][0]['name'] if track['artists'] else 'Unknown'
        track_name = track['name'] if track else 'Unknown'
        return artist_name, track_name
    except Exception as e:
        print(f"Failed to fetch data for {uri}: {str(e)}")
        return 'Unknown', 'Unknown'
    
df_lyrics['artist_name'], df_lyrics['track_name'] = zip(*df_lyrics['track_uri'].apply(get_track_details))

In [ ]:
import logging

# Setup basic configuration for logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def fetch_track_details(track_uris):
    try:
        tracks_info = sp.tracks(track_uris)['tracks']
        return [(track['artists'][0]['name'] if track['artists'] else 'Unknown', 
                 track['name'] if track else 'Unknown') for track in tracks_info]
    except Exception as e:
        print(f"Failed to process batch: {str(e)}")
        # Return a list of tuples with 'Unknown' values for artist and track names
        return [('Unknown', 'Unknown')] * len(track_uris)
    
batch_size = 50
artist_names = []
track_names = []

# Process in batches
for i in range(0, len(df_lyrics['track_uri']), batch_size):
    batch_uris = df_lyrics['track_uri'][i:i + batch_size].tolist()
    results = fetch_track_details(batch_uris)
    artist_names.extend([res[0] for res in results])
    track_names.extend([res[1] for res in results])

# Add the fetched data to the DataFrame
df_lyrics['artist_name'] = artist_names
df_lyrics['track_name'] = track_names


In [ ]:
def fetch_track_details(track_uris):
    try:
        tracks_info = sp.tracks(track_uris)['tracks']
        return [(track['artists'][0]['name'] if track['artists'] else 'Unknown', 
                 track['name'] if track else 'Unknown') for track in tracks_info]
    except Exception as e:
        print(f"Failed to process batch: {str(e)}")
        return [('Unknown', 'Unknown')] * len(track_uris)

# Define batch size and lists to hold data
batch_size = 50
artist_names = []
track_names = []

# Process in batches with progress updates and intermittent saving
for i in range(0, len(df_lyrics['track_uri']), batch_size):
    batch_uris = df_lyrics['track_uri'][i:i + batch_size].tolist()
    results = fetch_track_details(batch_uris)
    artist_names.extend([res[0] for res in results])
    track_names.extend([res[1] for res in results])
    print(f"Processed batch {i//batch_size + 1}/{(len(df_lyrics['track_uri']) + batch_size - 1)//batch_size}")

    # Intermediate save after each batch
    if i % 200 == 0:  # Change the condition as needed for more or fewer saves
        df_lyrics.loc[i:i+batch_size-1, 'artist_name'] = artist_names[-batch_size:]
        df_lyrics.loc[i:i+batch_size-1, 'track_name'] = track_names[-batch_size:]
        df_lyrics.to_csv('intermediate_df_lyrics.csv', index=False)
        print("Intermediate data saved.")

# Add the fetched data to the DataFrame
df_lyrics['artist_name'] = artist_names
df_lyrics['track_name'] = track_names

In [ ]:
# Apply batch processing
batch_size = 50
artist_names = []
track_names = []

for i in range(0, len(df_lyrics['track_uri']), batch_size):
    batch_uris = df_lyrics['track_uri'][i:i + batch_size].tolist()
    batch_results = fetch_tracks_details(batch_uris)
    artist_names.extend([res[0] for res in batch_results])
    track_names.extend([res[1] for res in batch_results])

df_lyrics['alb_name'] = artist_names
df_lyrics['trk_name'] = track_names

In [ ]:
# Save the updated DataFrame
df_lyrics.to_csv('df_lyrics_updated.csv', index=False)